In [1]:
# --- Configuración del Entorno y Carga de Datos Limpios ---

# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler

# Ignorar advertencias para una salida limpia
warnings.filterwarnings("ignore")

# Configurar el estilo y tamaño de las gráficas
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# --- 1. Cargar el conjunto de datos limpio ---
# Este archivo es el resultado del notebook EDA.ipynb
cleaned_data_path = 'datos/df_cleaned.csv'
try:
    # Asumimos que el df_cleaned fue guardado con su índice, por lo que lo usamos como index_col
    df_cleaned = pd.read_csv(cleaned_data_path, index_col=0)
    print(f"El conjunto de datos limpio se cargó exitosamente desde: '{cleaned_data_path}'.")
    print(f"Dimensiones del dataset: {df_cleaned.shape}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta: '{cleaned_data_path}'")
    print("Por favor, ejecute primero el notebook EDA.ipynb para generar este archivo.")
    df_cleaned = None

# --- 2. Mostrar información inicial del DataFrame ---
if df_cleaned is not None:
    print("\n--- Primeras 5 filas del conjunto de datos limpio ---")
    display(df_cleaned.head())
    print("\n--- Tipos de datos de las columnas ---")
    df_cleaned.info()

El conjunto de datos limpio se cargó exitosamente desde: 'datos/df_cleaned.csv'.
Dimensiones del dataset: (1068, 55)

--- Primeras 5 filas del conjunto de datos limpio ---


,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,ALC_mg/L,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
CLAVE,,,,,,,,,,,,,,,,,,,,,
DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,229.990,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,231.990,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,204.920,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,327.000,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,309.885,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI



--- Tipos de datos de las columnas ---
<class 'pandas.core.frame.DataFrame'>
Index: 1068 entries, DLAGU6 to OCRBR5109M1
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SITIO                 1068 non-null   object 
 1   ORGANISMO_DE_CUENCA   1068 non-null   object 
 2   ESTADO                1068 non-null   object 
 3   MUNICIPIO             1068 non-null   object 
 4   ACUIFERO              1068 non-null   object 
 5   SUBTIPO               1068 non-null   object 
 6   LONGITUD              1068 non-null   float64
 7   LATITUD               1068 non-null   float64
 8   PERIODO               1068 non-null   int64  
 9   ALC_mg/L              1068 non-null   float64
 10  CALIDAD_ALC           1068 non-null   object 
 11  CONDUCT_mS/cm         1068 non-null   float64
 12  CALIDAD_CONDUC        1068 non-null   object 
 13  SDT_M_mg/L            1068 non-null   float64
 14  CALIDAD_SDT_ra        106

### Sección 5: Ingeniería de Características y Preparación Final del Modelo

#### Resumen Ejecutivo de la Sección

Esta sección representa la fase final del preprocesamiento de datos, donde se aplican las decisiones estratégicas derivadas del Análisis Exploratorio de Datos (EDA). El objetivo fue construir el conjunto de datos final, `X_scaled`, optimizado para el entrenamiento del modelo de clustering. El pipeline consistió en tres etapas clave: selección de características, transformación no lineal y escalado.

#### Metodología y Ejecución

1.  **Selección de Características:** Basado en los hallazgos del EDA, se seleccionó un subconjunto de 15 características informativas. Este conjunto incluye las mediciones fisicoquímicas y bacteriológicas, junto con las coordenadas geográficas (`LONGITUD`, `LATITUD`). Se excluyeron deliberadamente los identificadores, las etiquetas de calidad predefinidas y la variable redundante `SDT_M_mg/L` (debido a su alta colinealidad con `CONDUCT_mS/cm`).

2.  **Transformación No Lineal:** Para mitigar el fuerte sesgo positivo identificado en las variables de medición, se aplicó una **transformación logarítmica (`np.log1p`)**. Esta operación comprime el rango de las características, reduce el impacto de los valores atípicos y resulta en distribuciones más simétricas, lo cual es fundamental para algoritmos basados en distancia como K-Means. Las coordenadas geográficas fueron excluidas de esta transformación.

3.  **Escalado de Características:** Como paso final, se aplicó un escalado mediante `StandardScaler` a todas las características seleccionadas. Este proceso estandariza cada característica para que tenga una media de cero y una desviación estándar de uno. El escalado es crucial para asegurar que todas las variables contribuyan de manera equitativa al cálculo de distancias del modelo, evitando que aquellas con magnitudes naturalmente mayores dominen el proceso de clustering.

#### Conclusión de la Sección

La ejecución de esta celda ha sido exitosa, culminando en la creación de la matriz de características `X_scaled` con dimensiones `(1068, 15)`. Esta matriz representa el conjunto de datos final, completamente preprocesado y optimizado. Está lista para ser utilizada como entrada directa en la siguiente fase del proyecto: el entrenamiento y la evaluación de los modelos de clustering.

In [2]:
# --- Sección 5: Ingeniería de Características y Preparación Final del Modelo ---

from sklearn.preprocessing import StandardScaler

if 'df_cleaned' in locals() and df_cleaned is not None:
    # --- 1. Selección de Características ---
    # Se elimina SDT_M_mg/L debido a su alta correlación con CONDUCT_mS/cm.
    features_for_clustering = [
        'LONGITUD', 'LATITUD', 'ALC_mg/L', 'CONDUCT_mS/cm', 'FLUORUROS_mg/L',
        'DUR_mg/L', 'COLI_FEC_NMP/100_mL', 'N_NO3_mg/L', 'AS_TOT_mg/L',
        'CD_TOT_mg/L', 'CR_TOT_mg/L', 'HG_TOT_mg/L', 'PB_TOT_mg/L',
        'MN_TOT_mg/L', 'FE_TOT_mg/L'
    ]
    df_model = df_cleaned[features_for_clustering].copy()

    # --- 2. Transformación de Datos (Logarítmica) ---
    # No transformamos Latitud y Longitud ya que no presentan el mismo tipo de sesgo.
    cols_to_transform = [col for col in features_for_clustering if col not in ['LONGITUD', 'LATITUD']]

    print("Aplicando transformación logarítmica a las características sesgadas...")
    for col in cols_to_transform:
        df_model[col] = np.log1p(df_model[col])

    print("\n--- Datos después de la Transformación Logarítmica (primeras 5 filas) ---")
    display(df_model.head())

    # --- 3. Escalado de Características ---
    print("\nEscalando todas las características con StandardScaler...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_model)

    print(f"\nDimensiones de los datos finales escalados para el clustering: {X_scaled.shape}")
    print("Los datos están ahora completamente preprocesados y listos para los algoritmos de clustering.")

else:
    print("El DataFrame 'df_cleaned' no fue encontrado. Asegúrate de que los pasos anteriores se ejecutaron correctamente.")

Aplicando transformación logarítmica a las características sesgadas...

--- Datos después de la Transformación Logarítmica (primeras 5 filas) ---


,LONGITUD,LATITUD,ALC_mg/L,CONDUCT_mS/cm,FLUORUROS_mg/L,DUR_mg/L,COLI_FEC_NMP/100_mL,N_NO3_mg/L,AS_TOT_mg/L,CD_TOT_mg/L,CR_TOT_mg/L,HG_TOT_mg/L,PB_TOT_mg/L,MN_TOT_mg/L,FE_TOT_mg/L
CLAVE,,,,,,,,,,,,,,,
DLAGU6,-102.02210,22.20887,5.442374,6.846943,0.681378,5.369391,0.741937,1.645703,0.015972,0.002996,0.004988,0.0005,0.004988,0.001499,0.085352
DLAGU6516,-102.20075,21.99958,5.450996,6.411818,0.657416,5.226023,0.741937,1.909559,0.013311,0.002996,0.004988,0.0005,0.004988,0.001499,0.024693
DLAGU7,-102.28801,22.36685,5.327488,6.278521,1.031225,4.801715,0.741937,0.896008,0.036332,0.002996,0.004988,0.0005,0.004988,0.001499,0.024693
DLAGU9,-102.29449,22.18435,5.793014,6.532334,0.752783,5.302703,0.741937,0.814744,0.015283,0.002996,0.004988,0.0005,0.004988,0.001499,0.024693
DLBAJ107,-110.24480,23.45138,5.739423,7.518607,0.210504,6.169584,5.676754,2.813746,0.009950,0.002996,0.004988,0.0005,0.004988,0.001499,0.024693



Escalando todas las características con StandardScaler...

Dimensiones de los datos finales escalados para el clustering: (1068, 15)
Los datos están ahora completamente preprocesados y listos para los algoritmos de clustering.
